# AI vs Human 文本偵測模型訓練

本 Notebook 用於在 Google Colab 上訓練五個模型：
- TF-IDF + SVM
- TF-IDF + Logistic Regression
- BERT
- RoBERTa + LoRA
- Hybrid

## 📋 使用說明

1. **啟用 GPU**：執行階段 → 變更執行階段類型 → 選擇 GPU
2. **上傳資料**：執行上傳 cell，選擇 `AI_Human.csv`
3. **依序執行**：按照順序執行各個 cell
4. **下載模型**：訓練完成後下載模型檔案



## 1. 環境設置


In [ ]:
# 安裝必要的套件
!pip install -q pandas numpy scikit-learn nltk transformers torch peft tqdm

# 下載 NLTK 資料
import nltk
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)

print("✅ 環境設置完成！")


## 2. 上傳資料

請上傳您的 `AI_Human.csv` 檔案


In [ ]:
from google.colab import files
import os

# 上傳檔案
uploaded = files.upload()

# 檢查上傳的檔案
for filename in uploaded.keys():
    print(f'✅ 已上傳: {filename}')
    if filename.endswith('.csv'):
        # 重新命名為 AI_Human.csv（如果需要）
        if filename != 'AI_Human.csv':
            os.rename(filename, 'AI_Human.csv')
            print(f'📝 已重新命名為 AI_Human.csv')

print("\n📂 當前目錄檔案:")
!ls -lh *.csv 2>/dev/null || echo "未找到 CSV 檔案"


## 3. 建立工具模組

### 3.1 預處理模組


In [ ]:
%%writefile preprocessing.py
"""
文本預處理工具
"""

import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def clean_text(text: str, remove_stopwords: bool = False) -> str:
    """清理文本"""
    if pd.isna(text):
        return ""
    
    text = str(text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    
    if remove_stopwords:
        stop_words = set(stopwords.words('english'))
        tokens = word_tokenize(text)
        text = ' '.join([token for token in tokens if token.lower() not in stop_words])
    
    return text

def preprocess_dataframe(df, text_column='text', remove_stopwords=False):
    """批量預處理 DataFrame"""
    df = df.copy()
    df[text_column] = df[text_column].apply(
        lambda x: clean_text(x, remove_stopwords=remove_stopwords)
    )
    return df

print("✅ 預處理模組已建立")


In [ ]:
import pandas as pd
import numpy as np
import pickle
import json
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from preprocessing import preprocess_dataframe

print("🚀 開始訓練 TF-IDF + SVM 模型...")

# 載入資料
print("📂 載入資料...")
df = pd.read_csv('AI_Human.csv', nrows=100000)
print(f"✅ 載入 {len(df)} 筆資料")

# 檢查欄位
if 'text' not in df.columns or 'generated' not in df.columns:
    if len(df.columns) >= 2:
        df.columns = ['text', 'generated'] + list(df.columns[2:])

df = df.dropna(subset=['text', 'generated'])
df['text'] = df['text'].astype(str)

# 轉換標籤
if df['generated'].dtype == bool:
    df['label'] = df['generated'].astype(int)
elif df['generated'].dtype == object:
    df['label'] = df['generated'].apply(
        lambda x: 1 if str(x).lower() in ['true', '1', 'ai', 'yes'] else 0
    )
else:
    df['label'] = df['generated'].astype(int)

print(f"📊 標籤分布: {df['label'].value_counts().to_dict()}")

# 預處理
print("🔧 預處理資料...")
df = preprocess_dataframe(df, text_column='text', remove_stopwords=False)

# 分割資料
X = df['text'].values
y = df['label'].values

X_train, X_temp, y_train, y_temp = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.67, random_state=42, stratify=y_temp
)

print(f"📊 資料分割: Train={len(X_train)}, Val={len(X_val)}, Test={len(X_test)}")

# TF-IDF 向量化
print("🔤 建立 TF-IDF 向量化器...")
vectorizer = TfidfVectorizer(
    max_features=5000,
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_val_tfidf = vectorizer.transform(X_val)
X_test_tfidf = vectorizer.transform(X_test)

print(f"✅ TF-IDF 特徵維度: {X_train_tfidf.shape[1]}")

# 訓練模型
print("🏋️  訓練 SVM (C=1.0, kernel=rbf)...")
print("⏳ 這可能需要 5-30 分鐘...")

start_time = time.time()
model = SVC(C=1.0, kernel='rbf', probability=True, random_state=42, verbose=True)
model.fit(X_train_tfidf, y_train)
elapsed_time = time.time() - start_time

print(f"✅ 訓練完成！耗時: {elapsed_time/60:.2f} 分鐘")

# 評估
print("📊 評估模型...")
y_val_pred = model.predict(X_val_tfidf)
val_accuracy = accuracy_score(y_val, y_val_pred)
print(f"✅ 驗證集準確率: {val_accuracy:.4f}")

y_test_pred = model.predict(X_test_tfidf)
test_accuracy = accuracy_score(y_test, y_test_pred)
print(f"✅ 測試集準確率: {test_accuracy:.4f}")

# 儲存模型
print("💾 儲存模型...")
!mkdir -p models/tfidf_svm

with open('models/tfidf_svm/model.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('models/tfidf_svm/vectorizer.pkl', 'wb') as f:
    pickle.dump(vectorizer, f)

# 儲存 metrics
metrics = {
    "model_name": "tfidf_svm",
    "baseline_accuracy": float(test_accuracy),
    "prompt_A_accuracy": float(test_accuracy),
    "prompt_B_accuracy": float(test_accuracy),
    "prompt_C_accuracy": float(test_accuracy),
    "validation_accuracy": float(val_accuracy),
    "test_accuracy": float(test_accuracy)
}

with open('models/tfidf_svm/metrics.json', 'w') as f:
    json.dump(metrics, f, indent=2)

print("✅ 模型已儲存至 models/tfidf_svm")

# 下載模型
from google.colab import files
!zip -r models_tfidf_svm.zip models/tfidf_svm/
files.download('models_tfidf_svm.zip')
print("\n✅ 模型檔案已準備下載！")
